In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
import os

In [78]:
#決定主要變數
img_size = 224
batch_size = 64
init_filter = 64
epoch = 25
cardinality = 0
features = '-revised'

#DataAugmentation
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=20
    )

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

#載入資料集
train_set = train_datagen.flow_from_directory(
    'train',
    target_size=(img_size, img_size),
    color_mode='rgb',
    classes=None,
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=None,
    save_to_dir=None,
    save_prefix='',
    save_format='png',
    follow_links=False,
    subset=None,
    interpolation='nearest',
    keep_aspect_ratio=False   
)


valid_set = test_datagen.flow_from_directory(
    'valid',
    target_size=(img_size, img_size),
    color_mode='rgb',
    classes=None,
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=None,
    save_to_dir=None,
    save_prefix='',
    save_format='png',
    follow_links=False,
    subset=None,
    interpolation='nearest',
    keep_aspect_ratio=False
)

#不會加入訓練，僅作為模型表現之參考
test_set = test_datagen.flow_from_directory(
    'test',
    target_size=(img_size, img_size),
    color_mode='rgb',
    classes=None,
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=None,
    save_to_dir=None,
    save_prefix='',
    save_format='png',
    follow_links=False,
    subset=None,
    interpolation='nearest',
    keep_aspect_ratio=False
)

Found 9866 images belonging to 11 classes.
Found 3430 images belonging to 11 classes.
Found 3347 images belonging to 11 classes.


In [79]:
#觀察許多結構有類似表現，因此特別寫一個函數出來，避免後續程式碼過於冗長
def transform_layer(x, filters, kernel_size):
    out = layers.Conv2D(filters, kernel_size, padding='same')(x)
    out = layers.BatchNormalization()(out)
    out = layers.ReLU()(out)
    return out

#題目中左邊（cardinality=0，同時是後續實作Resnet50中的第二種bottleneck）與中間之residual block（cardinality!=0）
def Residual_Block_1_2(inputs, filters, cardinality):
    fx = transform_layer(inputs, filters, 1)   
    if cardinality != 0:
        layers_split = list()
        for i in range(cardinality):
            splits = transform_layer(fx, filters, 3)
            layers_split.append(splits)
        fx = tf.keras.layers.concatenate(layers_split)        
    else:
        fx = transform_layer(fx, filters, 3)
        
    fx = layers.Conv2D(4*filters, 1, padding='same')(fx)
    fx = layers.BatchNormalization()(fx)
    outputs = layers.Add()([fx,inputs])
    outputs = layers.ReLU()(outputs)
    return outputs
    
#題目中右邊之residual block
def Residual_Block_3(inputs, filters):
    x = layers.Conv2D(filters,1)(inputs)    
    fx = transform_layer(inputs, filters/4,1)
    fx1 = transform_layer(fx, filters/4,3)
    fx2 = transform_layer(fx, filters/4,3)
    fx3 = transform_layer(fx, filters/4,1)
    
    fx1 = tf.keras.layers.GlobalAveragePooling2D()(fx1)
    fx4 = tf.keras.layers.Multiply()([fx1,fx2])
    fx4 = transform_layer(fx4, filters,1)
    
    fx5 = tf.keras.layers.concatenate([fx3,fx4])
    fx5 = layers.Conv2D(filters, 3, padding='same')(fx5)
    fx5 = layers.BatchNormalization()(fx5)
    
    outputs = layers.Add()([x,fx4,fx5])
    outputs = layers.ReLU()(outputs)
    
    return outputs   

#後續實作Resnet50中的第一種bottleneck，且加入strides為變數
def BTNK_1(inputs, filters, strides):
    x = layers.Conv2D(4*filters, 1, strides=strides)(inputs)
    x = layers.BatchNormalization()(x)
    fx = layers.Conv2D(filters, 1, strides=strides)(inputs)
    fx = layers.BatchNormalization()(fx)
    fx = layers.ReLU()(fx)  
    fx = transform_layer(fx, filters, 3)       
    fx = layers.Conv2D(4*filters, 1)(fx)
    fx = layers.BatchNormalization()(fx)
    outputs = layers.Add()([x,fx])
    outputs = layers.ReLU()(outputs)
    return outputs

In [80]:
#Resnet50_stage_input
inputs = keras.Input(shape=(img_size, img_size, 3))

filters = init_filter
#Resnet50_stage_0
x = layers.Conv2D(filters, 2, strides=2)(inputs)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = layers.MaxPool2D(pool_size=3, strides=2,padding='same')(x)


#Resnet50_stage_1
x = BTNK_1(x, filters, 1)
x = Residual_Block_1_2(x, filters, cardinality)
x = Residual_Block_1_2(x, filters, cardinality)

#Resnet50_stage_2
filters = filters*2
x = BTNK_1(x, filters, 2)
x = Residual_Block_1_2(x, filters, cardinality)
x = Residual_Block_1_2(x, filters, cardinality)
x = Residual_Block_1_2(x, filters, cardinality)

#Resnet50_stage_3
filters = filters*2
x = BTNK_1(x, filters, 2)
x = Residual_Block_1_2(x, filters, cardinality)
x = Residual_Block_1_2(x, filters, cardinality)
x = Residual_Block_1_2(x, filters, cardinality)
x = Residual_Block_1_2(x, filters, cardinality)
x = Residual_Block_1_2(x, filters, cardinality)

#Resnet50_stage_4
filters = filters*2
x = BTNK_1(x, filters, 2)
x = Residual_Block_1_2(x, filters, cardinality)
x = Residual_Block_1_2(x, filters, cardinality)

#Resnet50_stage_output
x = layers.AveragePooling2D()(x)
x = tf.keras.layers.Flatten()(x)
outputs = layers.Dense(11, activation='softmax')(x)
model = keras.Model(inputs=inputs,outputs=outputs)
model.summary()

Model: "model_17"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_25 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_985 (Conv2D)            (None, 112, 112, 64  832         ['input_25[0][0]']               
                                )                                                                 
                                                                                                  
 batch_normalization_950 (Batch  (None, 112, 112, 64  256        ['conv2d_985[0][0]']             
 Normalization)                 )                                                          

 re_lu_910 (ReLU)               (None, 56, 56, 64)   0           ['batch_normalization_959[0][0]']
                                                                                                  
 conv2d_995 (Conv2D)            (None, 56, 56, 256)  16640       ['re_lu_910[0][0]']              
                                                                                                  
 batch_normalization_960 (Batch  (None, 56, 56, 256)  1024       ['conv2d_995[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 add_296 (Add)                  (None, 56, 56, 256)  0           ['batch_normalization_960[0][0]',
                                                                  're_lu_908[0][0]']              
                                                                                                  
 re_lu_911

                                                                                                  
 add_299 (Add)                  (None, 28, 28, 512)  0           ['batch_normalization_970[0][0]',
                                                                  're_lu_917[0][0]']              
                                                                                                  
 re_lu_920 (ReLU)               (None, 28, 28, 512)  0           ['add_299[0][0]']                
                                                                                                  
 conv2d_1006 (Conv2D)           (None, 28, 28, 128)  65664       ['re_lu_920[0][0]']              
                                                                                                  
 batch_normalization_971 (Batch  (None, 28, 28, 128)  512        ['conv2d_1006[0][0]']            
 Normalization)                                                                                   
          

                                )                                 're_lu_926[0][0]']              
                                                                                                  
 re_lu_929 (ReLU)               (None, 14, 14, 1024  0           ['add_302[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1016 (Conv2D)           (None, 14, 14, 256)  262400      ['re_lu_929[0][0]']              
                                                                                                  
 batch_normalization_981 (Batch  (None, 14, 14, 256)  1024       ['conv2d_1016[0][0]']            
 Normalization)                                                                                   
                                                                                                  
 re_lu_930

 conv2d_1025 (Conv2D)           (None, 14, 14, 256)  262400      ['re_lu_938[0][0]']              
                                                                                                  
 batch_normalization_990 (Batch  (None, 14, 14, 256)  1024       ['conv2d_1025[0][0]']            
 Normalization)                                                                                   
                                                                                                  
 re_lu_939 (ReLU)               (None, 14, 14, 256)  0           ['batch_normalization_990[0][0]']
                                                                                                  
 conv2d_1026 (Conv2D)           (None, 14, 14, 256)  590080      ['re_lu_939[0][0]']              
                                                                                                  
 batch_normalization_991 (Batch  (None, 14, 14, 256)  1024       ['conv2d_1026[0][0]']            
 Normaliza

                                                                                                  
 re_lu_948 (ReLU)               (None, 7, 7, 512)    0           ['batch_normalization_1000[0][0]'
                                                                 ]                                
                                                                                                  
 conv2d_1036 (Conv2D)           (None, 7, 7, 512)    2359808     ['re_lu_948[0][0]']              
                                                                                                  
 batch_normalization_1001 (Batc  (None, 7, 7, 512)   2048        ['conv2d_1036[0][0]']            
 hNormalization)                                                                                  
                                                                                                  
 re_lu_949 (ReLU)               (None, 7, 7, 512)    0           ['batch_normalization_1001[0][0]'
          

In [81]:
run_name = 'img_size-'+str(img_size)+'-batch_size-'+str(batch_size)+'-init_filter-'+str(init_filter)+'-epoch-'+str(epoch)+features
model_dir = 'online_test/models' 
os.makedirs(model_dir,exist_ok=True) 

# 儲存訓練記錄檔
logfiles = 'online_test-logs/{}'.format(run_name)
model_cbk = keras.callbacks.TensorBoard(log_dir=logfiles, 
                                        histogram_freq=1)
# 儲存最好的網路模型權重
modelfiles = model_dir + '/{}-best-model.online_test'.format(run_name)
model_mckp = keras.callbacks.ModelCheckpoint(modelfiles, 
                                             monitor='val_categorical_accuracy', 
                                             save_best_only=True, 
                                             mode='max')

# 設定訓練使用的優化器、損失函數和指標函數
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=[keras.metrics.CategoricalAccuracy()])

# 訓練網路模型
model.fit(train_set,
          epochs=epoch, 
          validation_data=valid_set,
          callbacks=[model_cbk, model_mckp])


Epoch 1/25


2022-09-05 18:21:51.091428: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


155/155 [==============================] - ETA: 0s - loss: 3.1790 - categorical_accuracy: 0.2103

2022-09-05 18:29:56.084305: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


INFO:tensorflow:Assets written to: online_test/models/img_size-224-batch_size-64-init_filter-64-epoch-25-revised-best-model.online_test/assets


...

INFO:tensorflow:Assets written to: online_test/models/img_size-224-batch_size-64-init_filter-64-epoch-25-revised-best-model.online_test/assets


155/155 [==============================] - 470s 3s/step - loss: 1.2607 - categorical_accuracy: 0.5703 - val_loss: 1.7214 - val_categorical_accuracy: 0.4671
Epoch 23/25
155/155 [==============================] - ETA: 0s - loss: 1.2116 - categorical_accuracy: 0.5811

INFO:tensorflow:Assets written to: online_test/models/img_size-224-batch_size-64-init_filter-64-epoch-25-revised-best-model.online_test/assets


INFO:tensorflow:Assets written to: online_test/models/img_size-224-batch_size-64-init_filter-64-epoch-25-revised-best-model.online_test/assets


155/155 [==============================] - 467s 3s/step - loss: 1.2116 - categorical_accuracy: 0.5811 - val_loss: 1.6011 - val_categorical_accuracy: 0.5201
Epoch 24/25
155/155 [==============================] - 452s 3s/step - loss: 1.1779 - categorical_accuracy: 0.5905 - val_loss: 3.4645 - val_categorical_accuracy: 0.2668
Epoch 25/25
155/155 [==============================] - 451s 3s/step - loss: 1.1676 - categorical_accuracy: 0.5983 - val_loss: 1.9749 - val_categorical_accuracy: 0.4536


In [33]:
%load_ext tensorboard
%tensorboard --logdir online_test-logs

Reusing TensorBoard on port 6006 (pid 97103), started 1 day, 4:37:40 ago. (Use '!kill 97103' to kill it.)

In [82]:
#評估網路模型效果
model.load_weights(modelfiles)
loss, acc = model.evaluate(test_set)

2022-09-05 21:49:10.188697: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open online_test/models/img_size-224-batch_size-64-init_filter-64-epoch-25-revised-best-model.online_test: FAILED_PRECONDITION: online_test/models/img_size-224-batch_size-64-init_filter-64-epoch-25-revised-best-model.online_test; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


53/53 [==============================] - 36s 670ms/step - loss: 1.4985 - categorical_accuracy: 0.5372
